In [2]:
#importer les packages
import requests
import pandas as pd
import numpy as np
from tqdm.notebook import tqdm
tqdm.pandas()

In [3]:
rentree_sco=2023

In [4]:
liste_gentab=[]
for i in range(2015,rentree_sco+1,1):
    liste_gentab.append(pd.read_json(f"./POST_GENTAB/atlas{i}.json"))

In [4]:
df_tot=pd.concat(liste_gentab)

In [5]:
df_tot[(df_tot.RENTREE==2015)&(df_tot.REG_NOM!='Étranger')].EFFSDC.sum()

2579450

In [6]:
df_tot[(df_tot.RENTREE==2015)&(df_tot.REG_NOM!='Étranger')].EFFTOT.sum()

2620489

In [16]:
df_tot=df_tot[['RENTREE', 'ETABLISSEMENT', 'SECT', 'ETABLI', 'SIGLE_M', 'LIB1_M', 'LIB2_M', 'COMPOS', 'SIGLE_U', 'LIB1_U', 'LIB2_U', 'REG_ID', 'REG_NOM', 
 'ACA_ID', 'ACA_NOM', 'DEP_ID', 'DEP_NUM_NOM', 'UUCR_ID', 'UUCR_NOM', 'COM_CODE', 'COM_NOM', 'DEGETU', 'SEXE','EFFSDC']]

In [17]:
dict_a={'RENTREE':'rentree','EFFSDC':'effectifhdccpge','SECT':'secteur_etablissement','ETABLI':'id_etablissement','SIGLE_M':'sigle_etablissement',
        'LIB1_M':'libelle_etablissement_1','LIB2_M':'libelle_etablissement_2','COMPOS':'id_composante', 'ETABLISSEMENT':'categorie_etablissement',
        'LIB2_U':'libelle_composante_2','LIB1_U':'libelle_composante_1','SIGLE_U':'sigle_composante',
        'REG_ID':'reg_id','ACA_ID':'aca_id','DEP_ID':'dep_id','UUCR_ID':'uucr_id','COM_CODE':'com_code',
        'REG_NOM':'reg_nom','ACA_NOM':'aca_nom','DEP_NUM_NOM':'dep_num_nom','UUCR_NOM':'uucr_nom','COM_NOM':'com_nom','DEGETU':'degetu'}

In [18]:
df_tot=df_tot.rename(columns=dict_a)

In [19]:
df_tot.rentree.value_counts()

rentree
2023    64822
2022    61464
2021    59891
2020    30507
2019    29491
2018    28751
2017    28415
2016    26796
2015    26274
Name: count, dtype: int64

In [20]:
df_tot=df_tot[df_tot.effectifhdccpge!=0]

In [21]:
df_tot.rentree.value_counts()


rentree
2023    64729
2022    61383
2021    59816
2020    30463
2019    29446
2018    28702
2017    28373
2016    26757
2015    26244
Name: count, dtype: int64

In [22]:
df_tot['dont_femmes'] = df_tot.apply(lambda row: row['effectifhdccpge'] if row['SEXE'] == 2 else 0, axis = 1)
df_tot['dont_hommes'] = df_tot.apply(lambda row: row['effectifhdccpge'] if row['SEXE'] == 1 else 0, axis = 1)

In [23]:
df2=df_tot[df_tot.rentree.isin([2021,2022,2023])].groupby(['rentree', 'categorie_etablissement', 'secteur_etablissement',
       'id_etablissement', 'sigle_etablissement', 'libelle_etablissement_1',
       'libelle_etablissement_2', 'id_composante', 'sigle_composante',
       'libelle_composante_1', 'libelle_composante_2', 'reg_id', 'reg_nom',
       'aca_id', 'aca_nom', 'dep_id', 'dep_num_nom', 'uucr_id', 'uucr_nom',
       'com_code', 'com_nom', 'degetu']).agg({'effectifhdccpge':'sum','dont_hommes':'sum','dont_femmes':'sum'})

In [24]:
df1=df_tot[df_tot.rentree.isin([2015,2016,2017,2018,2019,2020])].groupby(['rentree', 'categorie_etablissement', 'secteur_etablissement',
       'id_etablissement', 'sigle_etablissement', 'libelle_etablissement_1',
       'libelle_etablissement_2', 'id_composante', 'sigle_composante',
       'libelle_composante_1', 'libelle_composante_2', 'reg_id', 'reg_nom',
       'aca_id', 'aca_nom', 'dep_id', 'dep_num_nom', 'uucr_id', 'uucr_nom',
       'com_code', 'com_nom']).agg({'effectifhdccpge':'sum','dont_hommes':'sum','dont_femmes':'sum'})

In [25]:
df1=df1.reset_index()
df2=df2.reset_index()

In [26]:
df=pd.concat([df1,df2])

In [27]:
df

,rentree,categorie_etablissement,secteur_etablissement,id_etablissement,sigle_etablissement,libelle_etablissement_1,libelle_etablissement_2,id_composante,sigle_composante,libelle_composante_1,...,dep_id,dep_num_nom,uucr_id,uucr_nom,com_code,com_nom,effectifhdccpge,dont_hommes,dont_femmes,degetu
0,2015,AUTRE UNIV,PR,0130247Y,FAC.THEO.REF.,FACULTE,JEAN CALVIN,0130247Y,FAC.THEO.REF.,FACULTE,...,D013,13 - Bouches-du-Rhône,UU00759,Marseille-Aix-en-Provence,13001,Aix-en-Provence,94,74,20,NaN
1,2015,AUTRE UNIV,PR,0310155A,ICT TOULOUSE,INSTITUT CATHOLIQUE TOULOUSE,,0311114T,FAC LETTRES,FACULTE LETTRES,...,D031,31 - Haute-Garonne,UU00758,Toulouse,31555,Toulouse,1173,288,885,NaN
2,2015,AUTRE UNIV,PR,0310155A,ICT TOULOUSE,INSTITUT CATHOLIQUE TOULOUSE,,0311243H,FAC THEOLOGIE,FACULTE THEOLOGIE,...,D031,31 - Haute-Garonne,UU00758,Toulouse,31555,Toulouse,1409,605,804,NaN
3,2015,AUTRE UNIV,PR,0310155A,ICT TOULOUSE,INSTITUT CATHOLIQUE TOULOUSE,,0311244J,FAC DRT CANON,FACULTE DROIT CANONIQUE,...,D031,31 - Haute-Garonne,UU00758,Toulouse,31555,Toulouse,54,41,13,NaN
4,2015,AUTRE UNIV,PR,0310155A,ICT TOULOUSE,INSTITUT CATHOLIQUE TOULOUSE,,0311245K,FAC PHILO,FACULTE PHILOSOPHIE,...,D031,31 - Haute-Garonne,UU00758,Toulouse,31555,Toulouse,198,99,99,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
62833,2023,veto,PU,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,ENVA,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,...,D094,94 - Val-de-Marne,UU00851,Paris,94046,Maisons-Alfort,328,62,266,2.0
62834,2023,veto,PU,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,ENVA,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,...,D094,94 - Val-de-Marne,UU00851,Paris,94046,Maisons-Alfort,158,37,121,3.0
62835,2023,veto,PU,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,ENVA,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,...,D094,94 - Val-de-Marne,UU00851,Paris,94046,Maisons-Alfort,132,35,97,4.0
62836,2023,veto,PU,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,ENVA,0940608A,ENV ALFORT,ECOLE NAT VETERINAIRE ALFORT,...,D094,94 - Val-de-Marne,UU00851,Paris,94046,Maisons-Alfort,103,26,77,5.0


In [28]:
dic_degetu = {'0':'Inférieur ou égal au baccalauréat','1':'BAC + 1','2':'BAC + 2','3':'BAC + 3','4':'BAC + 4','5':'BAC + 5','6':'BAC + 6 et plus'}
dic_etablissement = {'archi':"Écoles d'architecture",'art':'Écoles supérieures artistiques et culturelles','autre':'Autres écoles de spécialités diverses','AUTRE UNIV':"Autre établissements d'enseignement universitaire",'autre univ':"Autre établissements d'enseignement universitaire",'commerce':'Écoles de commerce, gestion et vente','ens':'Écoles normales supérieures','gd etab':'Grands établissements','inge':"Écoles d'ingénieurs",'inp ut':'Instituts nationaux polytechniques et universités de technologie','journalism':'Écoles de journalisme et écoles littéraires','journalisme':'Écoles de journalisme et écoles littéraires','jurid adm':'Écoles juridiques et administratives','lycee':'Lycées','paramedica':'Écoles paramédicales hors université','paramedical':'Écoles paramédicales hors université','social':'Écoles préparant aux fonctions sociales','univ priv':"Établissements d'enseignement universitaire privés",'universite':'Universités','veto':'Écoles vétérinaires','':np.nan}
dic_secteur = {'PR':'Privé','PU':'Public'}

In [29]:
df['degre_etudes']=df.loc[:,'degetu'].apply(lambda x: dic_degetu[str(int(x))] if pd.isna(x)==False else np.nan)

In [30]:
df.rentree.value_counts()

rentree
2023    21644
2022    20811
2021    20383
2019     7580
2020     7342
2018     7241
2017     7154
2016     6837
2015     6668
Name: count, dtype: int64

In [31]:
df.loc[:,'categorie_etablissement']=df.loc[:,'categorie_etablissement'].apply(lambda x: dic_etablissement[x] if (pd.isna(x)==False) else np.nan)

In [32]:
df.loc[:,'secteur_etablissement']=df.loc[:,'secteur_etablissement'].apply(lambda x: dic_secteur[x] if pd.isna(x)==False else np.nan)

In [33]:
df.rentree.value_counts()

rentree
2023    21644
2022    20811
2021    20383
2019     7580
2020     7342
2018     7241
2017     7154
2016     6837
2015     6668
Name: count, dtype: int64

In [34]:
df=df[['rentree', 'categorie_etablissement', 'secteur_etablissement',
       'id_etablissement', 'sigle_etablissement', 'libelle_etablissement_1',
       'libelle_etablissement_2', 'id_composante', 'sigle_composante',
       'libelle_composante_1', 'libelle_composante_2', 'reg_id', 'reg_nom',
       'aca_id', 'aca_nom', 'dep_id', 'dep_num_nom', 'uucr_id', 'uucr_nom',
       'com_code', 'com_nom', 'degetu', 'degre_etudes',
       'effectifhdccpge', 'dont_femmes', 'dont_hommes']]

In [35]:
df.to_json(f"./OUTPUT_OPENDATA/OD_atlas_etab.json", orient='records')
df.to_csv("./OUTPUT_OPENDATA/OD_atlas_etab.csv", sep=";", encoding="UTF-8", index=False)